In [5]:
from dataset import wiki_scraping
import numpy as np
import pickle
import json

In [16]:
with open('song_tf_idf.npy', 'rb') as f:
    song_tf_idf = np.load(f)
with open('word_to_index.pkl', 'rb') as f:
    word_to_index = pickle.load(f)
with open('song_to_index.pkl', 'rb') as f:
    song_to_index = pickle.load(f)
with open('index_to_song.pkl', 'rb') as f:
    index_to_song = pickle.load(f)

In [6]:
f = open('dataset/wiki_scraping/wiki_texts.json')
wiki_texts = json.load(f)

In [12]:
wiki_tf_idf = np.zeros((len(wiki_texts), song_tf_idf.shape[1]))
for i, city in enumerate(wiki_texts.keys()):
    txt = wiki_texts[city]
    for wrd in txt:
        if wrd in word_to_index:
            j = word_to_index[wrd]
            wiki_tf_idf[i, j] += 1

In [13]:
wiki_tf_idf = (wiki_tf_idf + 0.5) / (wiki_tf_idf.sum(axis=0) + 0.5)


In [14]:
loc_to_index = {cty:i for i, cty in enumerate(wiki_texts.keys())}

In [15]:
with open('wiki_tf_idf.npy', 'wb') as f:
    np.save(f, wiki_tf_idf)

In [32]:
def cos_sim(city, song):
    city_i = loc_to_index[city]
    song_i = song_to_index[song]
    city_vec = wiki_tf_idf[city_i, :]
    song_vec = song_tf_idf[song_i, :]
    denom = np.linalg.norm(city_vec) * np.linalg.norm(song_vec)
    num = city_vec @ song_vec
    return (num + 0.5) /  (denom + 0.5)

In [55]:
best_matches = []
for loc in loc_to_index:
    sim = cos_sim(loc, "A Milli")
    best_matches.append((loc, sim))

In [56]:
print("Toronto")
srtd = sorted(best_matches, key = lambda x: x[1], reverse=True)
for t in srtd[:10]:
    print("Song: ", t[0], "  Score: {:.6f}".format(t[1]))

Toronto
Song:  New York City   Score: 0.813875
Song:  London   Score: 0.811230
Song:  Budapest   Score: 0.811032
Song:  Toronto   Score: 0.810856
Song:  Mumbai   Score: 0.810727
Song:  Tokyo   Score: 0.808863
